In [1]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

In [2]:
df1 = pd.read_csv('optimizable_main.csv',index_col=[0])
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.44,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.44,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.266667


In [3]:
df2 =pd.read_csv('drug_dict_onehot.csv',index_col=[0])
df2.head()

,inpatient.number,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,722128,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,723327,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,723617,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,724385,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,725509,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [14]:
df2.drop(['inpatient.number'],axis=1,inplace=True)
df2.head()

,Atorvastatin calcium tablet,Benazepril hydrochloride tablet,Deslanoside injection,Digoxin tablet,Dobutamine hydrochloride injection,Furosemide injection,Furosemide tablet,Hydrochlorothiazide tablet,Isoprenaline Hydrochloride injection,Isosorbide Mononitrate Sustained Release tablet,Meglumine Adenosine Cyclophosphate for injection,Milrinone injection,Nitroglycerin injection,Shenfu injection,Spironolactone tablet,Torasemide tablet,Valsartan Dispersible tablet,sulfotanshinone sodium injection
0,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,1,0,1,1,0,1,1,0,0,1,0,1,0,0,1,1,1,0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0,0
4,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,1,0,0


In [5]:
q = df1

In [6]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
# fit on the dataset
imputer.fit(q)
xtrans = imputer.transform(q)

In [7]:
xtrans.shape

(1996, 144)

In [8]:
df_trans = pd.DataFrame(xtrans, columns =df1.columns)
df_trans.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,0.355118,0.045283,0.965333,0.530213,0.969251,0.422778,0.484,0.428571,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,0.341732,0.116981,0.826667,0.341277,0.829947,0.406111,0.524,0.639286,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,0.399606,0.075472,0.960000,0.390638,0.963102,0.492778,0.516,0.654762,1.0,0.266667


In [9]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split


for i in range(1,11):
    print('K:',i)
    q = df1

    imputer = KNNImputer(n_neighbors=i, weights='uniform', metric='nan_euclidean')
    # fit on the dataset
    imputer.fit(q)
    xtrans = imputer.transform(q)


    df_trans = pd.DataFrame(xtrans, columns =df1.columns)
    df_trans.head()

    X = df_trans
    y = df2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
    len(X), len(y_train), len(y_test)

    model = Sequential()
    model.add(Dense(30, input_dim=144, activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(18, activation='sigmoid'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    model.fit(X_train, y_train, epochs = 100,verbose=0)
    model.evaluate(X_test,y_test)


K: 1
13/13 [==============================] - 0s 831us/step - loss: 0.1604
K: 2
13/13 [==============================] - 0s 831us/step - loss: 0.1615
K: 3
13/13 [==============================] - 0s 748us/step - loss: 0.1567
K: 4
13/13 [==============================] - 0s 834us/step - loss: 0.1620
K: 5
13/13 [==============================] - 0s 831us/step - loss: 0.1598
K: 6
13/13 [==============================] - 0s 831us/step - loss: 0.1579
K: 7
13/13 [==============================] - 0s 831us/step - loss: 0.1577
K: 8
13/13 [==============================] - 0s 751us/step - loss: 0.1558
K: 9
13/13 [==============================] - 0s 831us/step - loss: 0.1586
K: 10
13/13 [==============================] - 0s 837us/step - loss: 0.1596


In [10]:
for i in range(11,21):
    print('K:',i)
    q = df1

    imputer = KNNImputer(n_neighbors=i, weights='uniform', metric='nan_euclidean')
    # fit on the dataset
    imputer.fit(q)
    xtrans = imputer.transform(q)


    df_trans = pd.DataFrame(xtrans, columns =df1.columns)
    df_trans.head()

    X = df_trans
    y = df2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
    len(X), len(y_train), len(y_test)

    model = Sequential()
    model.add(Dense(30, input_dim=144, activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(18, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam')

    model.fit(X_train, y_train, epochs = 100,verbose=0)
    model.evaluate(X_test,y_test)

K: 11
13/13 [==============================] - 0s 914us/step - loss: 0.1565
K: 12
13/13 [==============================] - 0s 831us/step - loss: 0.1561
K: 13
13/13 [==============================] - 0s 997us/step - loss: 0.1563
K: 14
13/13 [==============================] - 0s 748us/step - loss: 0.1565
K: 15
13/13 [==============================] - 0s 751us/step - loss: 0.1554
K: 16
13/13 [==============================] - 0s 834us/step - loss: 0.1547
K: 17
13/13 [==============================] - 0s 973us/step - loss: 0.1572
K: 18
13/13 [==============================] - 0s 748us/step - loss: 0.1570
K: 19
13/13 [==============================] - 0s 748us/step - loss: 0.1541
K: 20
13/13 [==============================] - 0s 936us/step - loss: 0.1567


In [11]:

for i in range(21,31):
    print('K:',i)
    q = df1

    imputer = KNNImputer(n_neighbors=i, weights='uniform', metric='nan_euclidean')
    # fit on the dataset
    imputer.fit(q)
    xtrans = imputer.transform(q)


    df_trans = pd.DataFrame(xtrans, columns =df1.columns)
    df_trans.head()

    X = df_trans
    y = df2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
    len(X), len(y_train), len(y_test)

    model = Sequential()
    model.add(Dense(30, input_dim=144, activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(18, activation='sigmoid'))
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    model.fit(X_train, y_train, epochs = 100,verbose=0)
    model.evaluate(X_test,y_test)

K: 21
13/13 [==============================] - 0s 748us/step - loss: 0.1585
K: 22
13/13 [==============================] - 0s 831us/step - loss: 0.1560
K: 23
13/13 [==============================] - 0s 840us/step - loss: 0.1573
K: 24
13/13 [==============================] - 0s 748us/step - loss: 0.1601
K: 25
13/13 [==============================] - 0s 831us/step - loss: 0.1586
K: 26
13/13 [==============================] - 0s 831us/step - loss: 0.1585
K: 27
13/13 [==============================] - 0s 838us/step - loss: 0.1570
K: 28
13/13 [==============================] - 0s 702us/step - loss: 0.1581
K: 29
13/13 [==============================] - 0s 856us/step - loss: 0.1587
K: 30
13/13 [==============================] - 0s 831us/step - loss: 0.1575


In [12]:
tf.random.set_seed = 42
q = df1

imputer = KNNImputer(n_neighbors=69, weights='uniform', metric='nan_euclidean')
# fit on the dataset
imputer.fit(q)
xtrans = imputer.transform(q)


df_trans = pd.DataFrame(xtrans, columns =df1.columns)
df_trans.head()

X = df_trans
y = df2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
len(X), len(y_train), len(y_test)

model = Sequential()
model.add(Dense(30, input_dim=144, activation='sigmoid'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, epochs = 100,verbose=0)
model.evaluate(X_test,y_test)

13/13 [==============================] - 0s 850us/step - loss: 0.1499


0.14986145496368408

In [ ]:
from skmultilearn.adapt import MLkNN
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
q = df1

imputer = KNNImputer(n_neighbors=18, weights='uniform', metric='nan_euclidean')
# fit on the dataset
imputer.fit(q)
xtrans = imputer.transform(q)


df_trans = pd.DataFrame(xtrans, columns =df1.columns)
df_trans.head()

X = df_trans
y = df2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
len(X), len(y_train), len(y_test)
X_train = X_train.to_numpy
y_train = y_train.to_numpy



classifier = MLkNN(k=3)

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

# print (clf.best_params_, clf.best_score_)
# model.fit(X_train, y_train, epochs = 100,verbose=0)
# model.evaluate(X_test,y_test)

In [ ]:
# using classifier chains
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [91]:
li = []
for column in df1:
    ct = df1[column].isna().sum()
    if(ct>1500):
        li.append(column)

print(li)
len(li)

['EA', 'tricuspid.valve.return.pressure', 'myoglobin', 'Inorganic.Phosphorus', 'serum.magnesium', 'erythrocyte.sedimentation.rate', 'homocysteine', 'apolipoprotein.A', 'apolipoprotein.B', 'lipoprotein']


10

In [92]:
li_new = {}

for i in li:
    ct = df1[i].isna().sum()
    li_new.setdefault(i,ct)
    
print(li_new)

{'EA': 1606, 'tricuspid.valve.return.pressure': 1817, 'myoglobin': 1599, 'Inorganic.Phosphorus': 1592, 'serum.magnesium': 1592, 'erythrocyte.sedimentation.rate': 1691, 'homocysteine': 1851, 'apolipoprotein.A': 1821, 'apolipoprotein.B': 1821, 'lipoprotein': 1821}


In [96]:
corr_mat = df1.corr()
corr_mat.to_csv("Correaltion.csv")

In [ ]:
rs = np.random.RandomState(0)
corr = df1.corr()
corr.style.background_gradient(cmap='coolwarm')


In [36]:
s = pd.read_csv("main_dataset_kNN.csv",index_col=[0])
s.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,map,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,...,measured.bicarbonate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,oxyhemoglobin,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.428309,0.046006,1.0,1.0,...,0.368876,0.073375,0.944444,0.400000,0.945187,0.420988,0.428889,0.408730,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.569853,0.053564,1.0,1.0,...,0.385827,0.132075,0.880000,0.187234,0.878342,0.394444,0.440000,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.514706,0.076992,1.0,0.5,...,0.496063,0.000000,0.973333,0.365957,0.981283,0.286111,0.440000,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.540441,0.049828,1.0,0.5,...,0.372375,0.087002,0.917778,0.335225,0.918449,0.435802,0.473333,0.549603,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.413603,0.054523,1.0,0.5,...,0.435477,0.084906,0.965185,0.412057,0.968731,0.386265,0.473333,0.581349,1.0,0.266667


In [5]:
df1.drop(['measured.bicarbonate','international.normalized.ratio','standard.bicarbonate','reduced.hemoglobin','oxyhemoglobin','measured.residual.base','standard.bicarbonate','hematocrit','measured.bicarbonate','neutrophil.count','total.carbon.dioxide','hematocrit.blood.gas','platelet.hematocrit','mean.corpuscular.volume','verbal.response','low.density.lipoprotein.cholesterol','tricuspid.valve.return.pressure','direct.bilirubin','movement','indirect.bilirubin','map','eosinophil.ratio','lactate.dehydrogenase','eye.opening'],inplace=True,axis=1)


In [6]:
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,Killip.grade,...,glucose.blood.gas,lactate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.046006,1.0,1.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.053564,1.0,1.0,0.000000,...,0.085784,0.109756,0.132075,0.880000,0.187234,0.394444,0.44,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.076992,1.0,0.5,0.333333,...,0.125000,0.085366,0.000000,0.973333,0.365957,0.286111,0.44,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.049828,1.0,0.5,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.054523,1.0,0.5,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.266667


In [7]:
df1.drop(['homocysteine'],axis=1,inplace=True)

In [8]:
df1.head()

,gender,body.temperature,pulse,respiration,systolic.blood.pressure,diastolic.blood.pressure,BMI,type.of.heart.failure,NYHA.cardiac.function.classification,Killip.grade,...,glucose.blood.gas,lactate,carboxyhemoglobin,oxygen.saturation,partial.oxygen.pressure,anion.gap,free.calcium,total.hemoglobin,GCS,ageCat
0,0.0,0.271429,0.333333,0.500000,0.480159,0.383562,0.046006,1.0,1.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
1,0.0,0.142857,0.363636,0.583333,0.595238,0.547945,0.053564,1.0,1.0,0.000000,...,0.085784,0.109756,0.132075,0.880000,0.187234,0.394444,0.44,0.226190,1.0,0.533333
2,1.0,0.200000,0.353535,0.555556,0.476190,0.547945,0.076992,1.0,0.5,0.333333,...,0.125000,0.085366,0.000000,0.973333,0.365957,0.286111,0.44,0.410714,1.0,0.800000
3,1.0,0.242857,0.651515,0.611111,0.484127,0.589041,0.049828,1.0,0.5,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.666667
4,0.0,0.271429,0.505051,0.555556,0.376984,0.445205,0.054523,1.0,0.5,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.266667


In [9]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

In [16]:
q = df1
i = 18
imputer = KNNImputer(n_neighbors=i, weights='uniform', metric='nan_euclidean')
# fit on the dataset
imputer.fit(q)
xtrans = imputer.transform(q)


df_trans = pd.DataFrame(xtrans, columns =df1.columns)
df_trans.head()

X = df_trans
y = df2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
len(X), len(y_train), len(y_test)

model = Sequential()
model.add(Dense(512, input_dim=121, activation='relu'))
model.add(Dense(256,  activation='relu'))
model.add(Dense(128,  activation='relu'))
model.add(Dense(64,  activation='relu'))
model.add(Dense(32,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss=tf.nn.softmax_cross_entropy_with_logits,metrics=['accuracy','mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(X_train, y_train, epochs = 500,verbose=1)
score = model.evaluate(X_test,y_test)

Epoch 1/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9486 - accuracy: 0.0508 - mse: 0.5232
Epoch 2/500
50/50 [==============================] - 0s 3ms/step - loss: 17.9541 - accuracy: 0.2525 - mse: 0.5967
Epoch 3/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.2957 - mse: 0.5967
Epoch 4/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.2976 - mse: 0.5967
Epoch 5/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.3014 - mse: 0.5967
Epoch 6/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.3033 - mse: 0.5967
Epoch 7/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.3039 - mse: 0.5967
Epoch 8/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accuracy: 0.3058 - mse: 0.5967
Epoch 9/500
50/50 [==============================] - 0s 2ms/step - loss: 17.9541 - accur

In [58]:
print(score)
#print(score[0],score[1])

1.0821259021759033


In [90]:
X = s
y = df2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
len(X), len(y_train), len(y_test)

model = Sequential()
model.add(Dense(30, input_dim=144, activation='relu'))
model.add(Dense(30,  activation='relu'))
model.add(Dense(18, activation='sigmoid'))
model.compile(loss=tf.compat.v1.losses.sigmoid_cross_entropy
,metrics=['accuracy','mse'], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(X_train, y_train, epochs = 500,batch_size= 512 ,verbose=0)
score = model.evaluate(X_test,y_test)

13/13 [==============================] - 0s 835us/step - loss: 0.6520 - accuracy: 0.0025 - mse: 0.2127
